In [0]:
%run ./conf_notebook

#### READ AND PLOT THE DATA

In [0]:
df_train = ps.read_csv(train_1_csv_file_path)
df_train = df_train.drop("_c0", axis=1)
df_train = df_train.to_pandas()
df_train = df_train.reset_index(drop=True)

X_train_ = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)
y_train_ = y_categorical

In [0]:
X_train_

,proto_encoded,state_encoded,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service_encoded,sload,dload,spkts,dpkts,swin,dwin,stcpb,dtcpb,smean,dmean,trans_depth,response_body_len,sjit,djit,rate,sinpkt,dinpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,-1.0,0.0,0.179469,-0.131902,0.007260,-0.010417,0.892857,0.000000,0.0,0.000000,-0.009738,0.253129,0.4,0.2,1.0,1.0,0.324406,1.150945,-0.697674,-0.011236,0.0,0.0,0.012007,0.102883,-0.025220,0.435473,0.163928,0.000000,0.000000,0.000000,0.0,-1.0,0.0,0.0,0.0,-0.4,-0.3,-0.166667,-0.285714,0.00,0.0,-0.181818
1,-1.0,0.0,0.970450,0.233129,37.976407,-1.000000,0.884921,0.666667,8.5,0.000000,-0.009803,18.032918,1.2,3.6,1.0,1.0,0.739772,1.608104,-0.488372,11.932584,0.0,0.0,0.024441,12.068622,-0.025185,0.900023,0.302174,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,3.8,0.2,-0.166667,-0.285714,0.00,0.0,-0.090909
2,-1.0,0.0,2.427244,-0.050613,11.816697,-1.000000,0.884921,0.333333,3.0,0.000000,-0.009880,2.136200,0.6,1.4,1.0,1.0,1.104087,1.548390,-0.627907,8.764045,0.0,0.0,6.835484,99.320499,-0.025700,4.199465,2.012246,1.708847,2.641310,1.296432,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.000000,-0.142857,0.00,0.0,0.000000
3,-1.0,0.0,2.514830,0.151840,0.549909,-1.000000,0.884921,0.333333,1.5,0.141403,-0.009867,0.068652,1.0,1.0,1.0,1.0,0.577632,0.547346,-0.488372,0.224719,0.0,0.0,0.103084,43.410362,-0.025704,2.766997,1.767374,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,-0.4,-0.3,0.000000,-0.142857,0.00,0.0,0.000000
4,-1.0,0.0,0.670406,0.079755,0.094374,0.000000,0.884921,0.666667,0.5,0.000000,-0.009801,0.091221,0.8,0.4,1.0,1.0,1.271038,1.033172,-0.465116,0.011236,0.0,0.0,0.961223,1.007099,-0.025546,0.860772,1.481864,1.960584,3.057719,1.471084,0.0,0.0,0.0,0.0,0.0,3.8,3.5,0.000000,-0.142857,0.25,0.0,3.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.0,1.0,-0.002355,-0.242331,-0.148820,0.000000,-0.115079,0.000000,0.0,1.000000,0.560186,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,-0.372093,-0.494382,0.0,0.0,0.000000,0.000000,0.863309,-0.004909,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,1.9,2.0,3.666667,3.000000,5.75,6.0,1.909091
175337,-1.0,0.0,0.754692,0.145706,0.172414,0.000000,0.884921,0.666667,0.5,0.000000,-0.009798,0.124133,0.8,0.6,1.0,1.0,1.835898,1.804430,-0.255814,0.000000,0.0,0.0,1.480554,1.045109,-0.025544,0.981350,1.311864,1.518608,1.585654,1.607593,0.0,0.0,0.0,0.0,0.0,-0.4,-0.3,-0.166667,-0.285714,0.00,0.0,-0.090909
175338,0.0,1.0,-0.002355,-0.242331,-0.148820,0.000000,-0.115079,0.000000,0.0,1.000000,0.560186,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,-0.372093,-0.494382,0.0,0.0,0.000000,0.000000,0.863309,-0.004909,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.7,0.8,0.166667,0.000000,0.50,1.0,0.909091
175339,0.0,1.0,-0.002355,-0.242331,-0.148820,0.000000,-0.115079,0.000000,0.0,1.000000,0.560186,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,-0.372093,-0.494382,0.0,0.0,0.000000,0.000000,0.863309,-0.004909,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,2.5,2.6,4.666667,3.857143,7.25,6.5,2.454545


In [0]:
df_test = ps.read_csv(test_1_csv_file_path)
df_test = df_test.drop("_c0", axis=1)
df_test = df_test.to_pandas()
df_test = df_test.reset_index(drop=True)

X_test_ = df_test.drop('attack_cat', axis=1)  
y = df_test['attack_cat']

y_encoded = label_encoder.transform(y)
y_categorical = to_categorical(y_encoded)
y_test_ = y_categorical

In [0]:
X_test_

,proto_encoded,state_encoded,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service_encoded,sload,dload,spkts,dpkts,swin,dwin,stcpb,dtcpb,smean,dmean,trans_depth,response_body_len,sjit,djit,rate,sinpkt,dinpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0.000000,1.0,-0.002352,0.050613,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,2.019491,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,4.069767,-0.494382,0.0,0.0,0.000000,0.000000,0.701650,-0.004873,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.3,-0.2,-0.166667,-0.285714,0.00,0.0,-0.090909
1,0.000000,1.0,-0.002356,1.021472,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,9.902808,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,18.790698,-0.494382,0.0,0.0,0.000000,0.000000,0.974449,-0.004927,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.3,-0.2,-0.166667,-0.285714,0.00,0.0,-0.090909
2,0.000000,1.0,-0.002361,0.489264,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,9.603514,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,10.720930,-0.494382,0.0,0.0,0.000000,0.000000,1.574607,-0.004982,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.2,-0.1,-0.166667,-0.285714,0.00,0.0,0.000000
3,0.000000,1.0,-0.002359,0.360429,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,6.741094,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,8.767442,-0.494382,0.0,0.0,0.000000,0.000000,1.307870,-0.004964,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.2,-0.1,0.000000,-0.142857,0.25,0.0,0.000000
4,0.000000,1.0,-0.002353,1.300613,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,9.558507,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,23.023256,-0.494382,0.0,0.0,0.000000,0.000000,0.774397,-0.004891,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.2,-0.1,0.000000,-0.142857,0.25,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000000,1.0,-0.002361,-0.250000,-0.148820,0.000000,-0.115079,0.000000,0.0,0.0,0.926240,-0.051967,0.0,-0.2,0.0,0.0,0.000000,0.000000,-0.488372,-0.494382,0.0,0.0,0.000000,0.000000,1.574607,-0.004982,-0.000118,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,-0.4,-0.3,0.000000,-0.142857,0.00,0.0,-0.090909
82328,-1.000000,0.0,1.653321,13.521472,0.172414,0.000000,0.884921,2.333333,0.5,0.0,-0.008501,0.028554,1.8,0.6,1.0,1.0,0.559588,1.716222,19.302326,0.000000,0.0,0.0,1.909100,1.660838,-0.025618,1.008186,2.814604,2.645164,4.305957,1.876754,0.0,0.0,0.0,0.0,0.0,-0.4,-0.2,0.000000,0.000000,0.00,0.0,-0.181818
82329,-2.895909,1.0,-0.002368,-0.294479,-0.148820,-1.322917,-0.115079,0.000000,0.0,0.0,-0.009898,-0.051967,-0.1,-0.2,0.0,0.0,0.000000,0.000000,-0.627907,-0.494382,0.0,0.0,0.000000,0.000000,-0.025813,1087.971739,-0.000118,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0,-0.4,-0.3,-0.166667,-0.285714,0.00,0.0,-0.181818
82330,-2.895909,1.0,-0.002368,-0.294479,-0.148820,-1.322917,-0.115079,0.000000,0.0,0.0,-0.009898,-0.051967,-0.1,-0.2,0.0,0.0,0.000000,0.000000,-0.627907,-0.494382,0.0,0.0,0.004359,0.000000,-0.025813,1087.971956,-0.000118,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0,-0.4,-0.3,-0.166667,-0.285714,0.00,0.0,-0.181818


In [0]:
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# Print the mapping
print(label_mapping)

{'Analysis': 0, 'Backdoor': 1, 'DoS': 2, 'Exploits': 3, 'Fuzzers': 4, 'Generic': 5, 'Normal': 6, 'Reconnaissance': 7, 'Shellcode': 8, 'Worms': 9}


In [0]:
# # Define training data
# train_partition = 4

# train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
# df_train = spark.read.parquet(train_path)
# df_train = ps.DataFrame(df_train).to_pandas()
# df_train = df_train.reset_index(drop=True)

# print(df_train["attack_cat"].value_counts())

# train_counts = df_train['attack_cat'].value_counts()

# # Define the desired order of the class labels
# desired_order = ['Normal', 'Generic', 'Exploits', 'Fuzzers', 'DoS', 'Reconnaissance', 'Analysis', 'Backdoor', 'Shellcode', 'Worms']

# # Reindex the train_counts Series to match the desired order
# train_counts = train_counts.reindex(desired_order)

# # Create an interactive bar plot for the training data with steel blue color and thinner bars
# fig = px.bar(train_counts, x=train_counts.index, y=train_counts.values, labels={'x': 'Attack Category', 'y': 'Number of records'},
#              color_discrete_sequence=['steelblue'])

# # Update the bar width (e.g., 0.4 for thinner bars)
# fig.update_traces(width=0.4)

# fig.update_layout(
#     title=f'Training Data {train_partition}. Total number of records = {df_train.shape[0]}',
#     height=500,  # Set the height of the graph
#     width=800    # Set the width of the graph
# )

# # Show the plot
# fig.show()

#### FUNCTIONS

In [0]:
def train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[]):

  # BASELINE ANN --------------------------------------------------------------------------------------------

  early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    verbose=1,           
    restore_best_weights=True  
  )

  model = Sequential()
  model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  history = model.fit(
      X_train, y_train,
      validation_data=(X_test, y_test),
      epochs=50,
      batch_size=12,
      callbacks=[early_stopping] 
  )
  
  class_labels = label_encoder.classes_
  all_class_indices = range(len(class_labels))
  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Test accuracy: {accuracy*100:.2f}%')

  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_true_classes = np.argmax(y_test, axis=1)

  report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
  print(report)

  # FINE-TUNE ANN --------------------------------------------------------------------------------------------

  df_train = ps.read_csv(train_1_csv_file_path).to_pandas()
  df_train = df_train.drop("_c0", axis=1)
  df_train = df_train.reset_index(drop=True)
  df_train = df_train[~df_train["attack_cat"].isin(exclude_categories)]

  X = df_train.drop('attack_cat', axis=1)  
  y = df_train['attack_cat']

  y_encoded = label_encoder.transform(y)
  print(np.unique(y_encoded))
  y_categorical = to_categorical(y_encoded)
  print(y_categorical.shape)

  X_train, X_test, y_train, y_test = train_test_split(
      X, y_categorical, test_size=0.2, random_state=42, stratify=y
  )

  history = model.fit(
      X_train, y_train,
      validation_data=(X_test, y_test),
      epochs=50,
      batch_size=12,
      callbacks=[early_stopping]  
  )

  class_labels = label_encoder.classes_
  all_class_indices = range(len(class_labels))

  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Test accuracy: {accuracy*100:.2f}%')

  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_true_classes = np.argmax(y_test, axis=1)

  report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
  print(report)

  return model

In [0]:
def train_base_ann_and_fine_tune_final(X_train, y_train, y_categorical, epochs_base, epochs_fined, exclude_categories=[]):

  # BASELINE ANN --------------------------------------------------------------------------------------------

  model = Sequential()
  model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  history = model.fit(
      X_train, y_train,
      epochs=epochs_base,
      batch_size=12
  )

  # FINE-TUNE ANN --------------------------------------------------------------------------------------------

  df_train = ps.read_csv(train_1_csv_file_path).to_pandas()
  df_train = df_train.drop("_c0", axis=1)
  df_train = df_train.reset_index(drop=True)
  df_train = df_train[~df_train["attack_cat"].isin(exclude_categories)]

  X = df_train.drop('attack_cat', axis=1)  
  y = df_train['attack_cat']

  y_encoded = label_encoder.transform(y)
  print(np.unique(y_encoded))
  y_categorical = to_categorical(y_encoded)
  print(y_categorical.shape)

  X_train, y_train, = X, y_categorical

  history = model.fit(
      X_train, y_train,
      epochs=epochs_fined,
      batch_size=12
  )

  return model

#### ANN 0

In [0]:
train_partition = 0

train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=["Shellcode", "Worms"])


[3 4 5 6 7]
(38939, 8)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2596/2596 ━━━━━━━━━━━━━━━━━━━━ 3s 976us/step - accuracy: 0.9925 - loss: 0.0614 - val_accuracy: 0.9996 - val_loss: 0.0015
Epoch 2/50
2596/2596 ━━━━━━━━━━━━━━━━━━━━ 2s 932us/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9999 - val_loss: 9.0481e-04
Epoch 3/50
2596/2596 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9999 - val_loss: 5.5837e-04
Epoch 4/50
 314/2596 ━━━━━━━━━━━━━━━━━━━━ 1s 821us/step - accuracy: 1.0000 - l

*** WARNING: max output size exceeded, skipping output. ***

2596/2596 ━━━━━━━━━━━━━━━━━━━━ 2s 915us/step - accuracy: 1.0000 - loss: 7.5811e-05 - val_accuracy: 1.0000 - val_loss: 4.4535e-05
Epoch 12/50
2596/2596 ━━━━━━━━━━━━━━━━━━━━ 2s 899us/step - accuracy: 1.0000 - loss: 2.7537e-05 - val_accuracy: 0.9997 - val_loss: 5.3329e-04
Epoch 13/50
2596/2596 ━━━━━━━━━━━━━━━━━━━━ 3s 912us/step - accuracy: 1.0000 - loss: 1.1695e-04 - val_accuracy: 1.0000 - val_loss: 2.1691e-06
Epoch 13: early stopping
Restoring model weights from the

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7]
(174078, 8)
Epoch 1/50
 85

*** WARNING: max output size exceeded, skipping output. ***

11606/11606 ━━━━━━━━━━━━━━━━━━━━ 10s 900us/step - accuracy: 0.7873 - loss: 0.5151 - val_accuracy: 0.7931 - val_loss: 0.5011
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 597us/step - accuracy: 0.7626 - loss: 0.6029
Test accuracy: 76.40%
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 622us/step
                precision    recall  f1-score   support

      Analysis     1.0000    0.0175    0.0344       400
      Backdoor     0.0000    0.0000    0.0000       349
           DoS     0.2905    0.0285    0.0520      2453
      Exploits     0.5375    0.8841    0.6685      6679
       Fuzzers     0.6052    0.6582    0.6306      3637
       Generic     0.9997    0.9778    0.9886      8000
        Normal     0.9255    0.7814    0.8474     11200
Reconnaissance     0.7032    0.7860    0.7423      2098
     Shellcode     0.0000    0.0000

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=["Shellcode", "Worms"])

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
3245/3245 ━━━━━━━━━━━━━━━━━━━━ 3s 732us/step - accuracy: 0.9803 - loss: 0.0726
Epoch 2/50
3245/3245 ━━━━━━━━━━━━━━━━━━━━ 2s 746us/step - accuracy: 0.9998 - loss: 8.1801e-04
Epoch 3/50
1937/3245 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.9999 - loss

*** WARNING: max output size exceeded, skipping output. ***

14507/14507 ━━━━━━━━━━━━━━━━━━━━ 10s 717us/step - accuracy: 0.8021 - loss: 0.4778


In [0]:

y_pred_0 = model.predict(X_train_)

soft_classification_0 = pd.DataFrame(y_pred_0, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_0['class_prob_8'] = 0
soft_classification_0['class_prob_9'] = 0
soft_classification_0['ANN'] = 0
soft_classification_0 = soft_classification_0.reset_index()

soft_classification_0.head()

y_pred_classes = np.argmax(y_pred_0, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 609us/step
                precision    recall  f1-score   support

      Analysis     0.6936    0.0815    0.1459      2000
      Backdoor     0.7007    0.1140    0.1961      1746
           DoS     0.5734    0.0439    0.0816     12264
      Exploits     0.5665    0.9443    0.7081     33393
       Fuzzers     0.7205    0.6644    0.6913     18184
       Generic     0.9996    0.9779    0.9886     40000
        Normal     0.9088    0.8603    0.8839     56000
Reconnaissance     0.8759    0.7766    0.8233     10491
     Shellcode     0.0000    0.0000    0.0000      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7982    175341
     macro avg     0.6039    0.4463    0.4519    175341
  weighted avg     0.8083    0.7982    0.7730    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_0_test = model.predict(X_test_)

soft_classification_0_test = pd.DataFrame(y_pred_0_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_0_test['class_prob_8'] = 0
soft_classification_0_test['class_prob_9'] = 0
soft_classification_0_test['ANN'] = 0
soft_classification_0_test = soft_classification_0_test.reset_index()

soft_classification_0_test.head()

y_pred_classes = np.argmax(y_pred_0_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 606us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.0455    0.0395    0.0423       583
           DoS     0.4180    0.0761    0.1287      4089
      Exploits     0.5010    0.9060    0.6452     11132
       Fuzzers     0.3120    0.5056    0.3859      6062
       Generic     0.9991    0.9545    0.9763     18871
        Normal     0.9526    0.7567    0.8434     37000
Reconnaissance     0.8459    0.8209    0.8332      3496
     Shellcode     0.0000    0.0000    0.0000       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7575     82332
     macro avg     0.4074    0.4059    0.3855     82332
  weighted avg     0.8048    0.7575    0.7605     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 4

In [0]:
train_partition = 4

train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[0 1 2 3 4 5 6 7 9]
(29444, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1963/1963 ━━━━━━━━━━━━━━━━━━━━ 3s 937us/step - accuracy: 0.4283 - loss: 135.2664 - val_accuracy: 0.4474 - val_loss: 1.5613
Epoch 2/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 879us/step - accuracy: 0.4491 - loss: 30.8388 - val_accuracy: 0.4478 - val_loss: 2.0367
Epoch 3/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 875us/step - accuracy: 0.4489 - loss: 18.2838 - val_accuracy: 0.4485 - val_loss: 1.8077
Epoch 4/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 902us/step - accuracy: 0.4468 - loss: 13.9393 - val_accuracy: 0.4480 - val_loss: 1.5568
Epoch 5/50
 612/1963 ━━━━━━━━━━━━━━━━━━━━ 1s 743us/step - accuracy: 0.4571 - loss: 7.5159

*** WARNING: max output size exceeded, skipping output. ***

1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 897us/step - accuracy: 0.4539 - loss: 1.3889 - val_accuracy: 0.4495 - val_loss: 1.4100
Epoch 8/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 880us/step - accuracy: 0.4501 - loss: 2.9765 - val_accuracy: 0.4497 - val_loss: 1.4163
Epoch 9/50
1963/1963 ━━━━━━━━━━━━━━━━━━━━ 2s 

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8828/11690 ━━━━━━━━━━━━━━━━━━━━ 2s 7

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 862us/step - accuracy: 0.8034 - loss: 0.4890 - val_accuracy: 0.8022 - val_loss: 1.0614
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 15.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.8032 - loss: 0.4881
Test accuracy: 80.31%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 611us/step
                precision    recall  f1-score   support

      Analysis     0.6897    0.0500    0.0932       400
      Backdoor     0.6591    0.0831    0.1476       349
           DoS     0.5938    0.0310    0.0589      2453
      Exploits     0.5907    0.8766    0.7058      6679
       Fuzzers     0.7344    0.6318    0.6793      3637
       Generic     0.9996    0.9772    0.9883      8000
        Normal     0.8661    0.9255    0.8949     11200
Reconnaissance     0.8621    0.7421    0.7976     

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2454/2454 ━━━━━━━━━━━━━━━━━━━━ 3s 720us/step - accuracy: 0.4291 - loss: 78.2948
Epoch 2/50
2454/2454 ━━━━━━━━━━━━━━━━━━━━ 2s 736us/step - accuracy: 0.4432 - loss: 51.9784
Epoch 3/50
2454/2454 ━━━━━━━━━━━━━━━━━━━━ 2s 750us/step - accuracy: 0.4455 - loss: 8.6267
Epoch 4/50
1344/2454 ━━━━━━━━━━━━━━━━━━━━ 0s 713

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 11s 723us/step - accuracy: 0.7939 - loss: 0.5153


In [0]:
y_pred_1 = model.predict(X_train_)

soft_classification_1 = pd.DataFrame(y_pred_1, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_1['ANN'] = 4
soft_classification_1 = soft_classification_1.reset_index()

soft_classification_1.head()

y_pred_classes = np.argmax(y_pred_1, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 608us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.5052    0.1380    0.2168      1746
           DoS     0.4012    0.1652    0.2340     12264
      Exploits     0.5757    0.8864    0.6980     33393
       Fuzzers     0.6800    0.7610    0.7182     18184
       Generic     0.9997    0.9778    0.9887     40000
        Normal     0.9497    0.8190    0.8795     56000
Reconnaissance     0.8820    0.7728    0.8238     10491
     Shellcode     0.5626    0.7140    0.6293      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7962    175341
     macro avg     0.5556    0.5234    0.5188    175341
  weighted avg     0.8010    0.7962    0.7857    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_1_test = model.predict(X_test_)

soft_classification_1_test = pd.DataFrame(y_pred_1_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_1_test['ANN'] = 4
soft_classification_1_test = soft_classification_1_test.reset_index()

soft_classification_1_test.head()

y_pred_classes = np.argmax(y_pred_1_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 604us/step
                precision    recall  f1-score   support

      Analysis     0.0882    0.0044    0.0084       677
      Backdoor     0.0356    0.1012    0.0527       583
           DoS     0.3697    0.0555    0.0965      4089
      Exploits     0.4742    0.8517    0.6092     11132
       Fuzzers     0.2838    0.5982    0.3850      6062
       Generic     0.9944    0.9549    0.9742     18871
        Normal     0.9681    0.6482    0.7765     37000
Reconnaissance     0.8627    0.8144    0.8378      3496
     Shellcode     0.2524    0.7063    0.3719       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7107     82332
     macro avg     0.4329    0.4735    0.4112     82332
  weighted avg     0.8051    0.7107    0.7255     82332



#### ANN 2

In [0]:
train_partition = 2

train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[0 1 2 3 4 5 6 7 9]
(44676, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2979/2979 ━━━━━━━━━━━━━━━━━━━━ 3s 906us/step - accuracy: 0.9794 - loss: 9.5759 - val_accuracy: 0.9936 - val_loss: 0.0205
Epoch 2/50
2979/2979 ━━━━━━━━━━━━━━━━━━━━ 3s 868us/step - accuracy: 0.9938 - loss: 0.0231 - val_accuracy: 0.9942 - val_loss: 0.0192
Epoch 3/50
2951/2979 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.9941 - loss: 0.0520

*** WARNING: max output size exceeded, skipping output. ***

2979/2979 ━━━━━━━━━━━━━━━━━━━━ 3s 882us/step - accuracy: 0.9944 - loss: 0.0207 - val_accuracy: 0.9943 - val_loss: 0.0187
Epoch 8/50
2979/2979 ━━━━━━━━━━━━━━━━━━━━ 3s 860us/step - accuracy: 0.9941 - loss: 0.0334 - val_accuracy: 0.9942 - val_loss: 0.0179
Epoch 9/50
2979/2979 ━━━━━━━━━━━━━━━━━━━━ 3s 884us/step - accuracy: 0.9951 - loss: 0.0168 - val_accuracy: 0.9949 - val_loss: 0.0180
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - accuracy: 0.9951

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8840/11690 ━━━━━━━━━━━━━━━━━━━━ 2s 724us/step - accuracy: 0.7158 - loss: 5.3052

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 860us/step - accuracy: 0.7857 - loss: 0.6166 - val_accuracy: 0.7876 - val_loss: 0.5219
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step - accuracy: 0.7582 - loss: 0.6118
Test accuracy: 76.15%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 618us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       400
      Backdoor     0.0000    0.0000    0.0000       349
           DoS     0.4286    0.0049    0.0097      2453
      Exploits     0.5200    0.9603    0.6747      6679
       Fuzzers     0.6317    0.5296    0.5761      3637
       Generic     0.9995    0.9776    0.9884      8000
        N

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3723/3723 ━━━━━━━━━━━━━━━━━━━━ 3s 718us/step - accuracy: 0.9663 - loss: 0.8988
Epoch 2/50
3723/3723 ━━━━━━━━━━━━━━━━━━━━ 3s 713us/step - accuracy: 0.9935 - loss: 0.0304
Epoch 3/50
1590/3723 ━━━━━━━━━━━━━━━━━━━━ 1s 728us/step - accuracy: 0.9932 - loss: 0.0442

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 21s 731us/step - accuracy: 0.7903 - loss: 0.5302


In [0]:
y_pred_2 = model.predict(X_train_)

soft_classification_2 = pd.DataFrame(y_pred_2, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_2['ANN'] = 2
soft_classification_2 = soft_classification_2.reset_index()

soft_classification_2.head()

y_pred_classes = np.argmax(y_pred_2, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 613us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.9554    0.0613    0.1152      1746
           DoS     0.4446    0.0400    0.0733     12264
      Exploits     0.5870    0.8047    0.6788     33393
       Fuzzers     0.6902    0.7139    0.7019     18184
       Generic     0.9999    0.9772    0.9884     40000
        Normal     0.8391    0.8952    0.8662     56000
Reconnaissance     0.8690    0.7705    0.8168     10491
     Shellcode     0.5726    0.7096    0.6338      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7902    175341
     macro avg     0.5958    0.4972    0.4874    175341
  weighted avg     0.7758    0.7902    0.7635    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_2_test = model.predict(X_test_)

soft_classification_2_test = pd.DataFrame(y_pred_2_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_2_test['ANN'] = 2
soft_classification_2_test = soft_classification_2_test.reset_index()

soft_classification_2_test.head()

y_pred_classes = np.argmax(y_pred_2_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 610us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.5000    0.0292    0.0551       583
           DoS     0.1522    0.0171    0.0308      4089
      Exploits     0.4920    0.8180    0.6145     11132
       Fuzzers     0.2667    0.5016    0.3482      6062
       Generic     0.9999    0.8864    0.9398     18871
        Normal     0.8974    0.7462    0.8148     37000
Reconnaissance     0.8377    0.8164    0.8269      3496
     Shellcode     0.2598    0.7037    0.3795       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7250     82332
     macro avg     0.4406    0.4519    0.4009     82332
  weighted avg     0.7665    0.7250    0.7291     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 3

In [0]:
train_partition = 3

train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[0 1 2 3 4 5 6 7 9]
(16308, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1088/1088 ━━━━━━━━━━━━━━━━━━━━ 2s 986us/step - accuracy: 0.7230 - loss: 90.6956 - val_accuracy: 0.8286 - val_loss: 18.2012
Epoch 2/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step - accuracy: 0.7892 - loss: 20.9083 - val_accuracy: 0.8430 - val_loss: 2.4954
Epoch 3/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step - accuracy: 0.8231 - loss: 2.6310 - val_accuracy: 0.8326 - val_loss: 0.6187
Epoch 4/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 873us/step - accuracy: 0.8340 - loss: 0.5534 - val_accuracy: 0.8467 - val_loss: 0.5079
Epoch 5/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step - accuracy: 0.8513 - loss: 0.4806 - val_accuracy: 0.8516 - val_loss: 0.4933
Epoch 6/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - accuracy: 0.8576 - loss: 0.4552 - val_accuracy: 0.8486 - val_loss: 0.6103
Epoch 7/50
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 877us/step - accuracy: 0.8554 - loss: 0.4690 - val_accuracy: 0.8596 - val_loss: 0.4740
Epoch 8/50
 760/1088 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.8645 - loss: 0

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8969/11690 ━━━━━━━━━━━━━━━━━━━━ 1s 7

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 860us/step - accuracy: 0.7851 - loss: 0.5288 - val_accuracy: 0.7854 - val_loss: 0.5222
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 630us/step - accuracy: 0.7700 - loss: 0.5683
Test accuracy: 77.01%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step
                precision    recall  f1-score   support

      Analysis     0.6061    0.0500    0.0924       400
      Backdoor     0.6333    0.0544    0.1003       349
           DoS     0.3077    0.0065    0.0128      2453
      Exploits     0.5436    0.9206    0.6836      6679
       Fuzzers     0.6228    0.6800    0.6501      3637
       Generic     0.9996    0.9776    0.9885      8000
        Normal     0.8965    0.8107    0.8515     11200
Reconnaissance     0.8409    0.6549    0.7363      2

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1359/1359 ━━━━━━━━━━━━━━━━━━━━ 2s 696us/step - accuracy: 0.7454 - loss: 70.8463
Epoch 2/50
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.8021 - loss: 10.4602
Epoch 3/50
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.8387 - loss: 0.6003
Epoch 4/50
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 734us/step - accuracy: 0.8373 - loss: 0.5557
Epoch 5/50
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 710us/step - accuracy: 0.8402 - loss: 0.5315
Epoch 6/50
1359/1359 ━━━━━━━━━━━━━━━━━━━━ 1s 697us/step - accuracy: 0.8529 - loss: 0.5554
Epoch 7/50
 715/1359 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.8549 - loss: 0.4637

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 10s 709us/step - accuracy: 0.8019 - loss: 0.6012


In [0]:
y_pred_3 = model.predict(X_train_)

soft_classification_3 = pd.DataFrame(y_pred_3, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_3['ANN'] = 3
soft_classification_3 = soft_classification_3.reset_index()

y_pred_classes = np.argmax(y_pred_3, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 4s 667us/step
                precision    recall  f1-score   support

      Analysis     0.5620    0.0680    0.1213      2000
      Backdoor     0.7626    0.0865    0.1553      1746
           DoS     0.5865    0.0453    0.0842     12264
      Exploits     0.5933    0.8533    0.7000     33393
       Fuzzers     0.7552    0.5922    0.6639     18184
       Generic     0.9999    0.9776    0.9886     40000
        Normal     0.8403    0.9387    0.8868     56000
Reconnaissance     0.8876    0.7623    0.8202     10491
     Shellcode     0.6825    0.5958    0.6362      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.8010    175341
     macro avg     0.6670    0.4920    0.5056    175341
  weighted avg     0.8003    0.8010    0.7729    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_3_test = model.predict(X_test_)

soft_classification_3_test = pd.DataFrame(y_pred_3_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_3_test['ANN'] = 3
soft_classification_3_test = soft_classification_3_test.reset_index()

soft_classification_3_test.head()

y_pred_classes = np.argmax(y_pred_3_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 627us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.4259    0.0395    0.0722       583
           DoS     0.3657    0.0479    0.0848      4089
      Exploits     0.5263    0.8421    0.6477     11132
       Fuzzers     0.3182    0.4396    0.3692      6062
       Generic     0.9998    0.9489    0.9737     18871
        Normal     0.9068    0.8226    0.8626     37000
Reconnaissance     0.8365    0.8092    0.8226      3496
     Shellcode     0.4418    0.6323    0.5201       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7733     82332
     macro avg     0.4821    0.4582    0.4353     82332
  weighted avg     0.7900    0.7733    0.7676     82332



#### ANN 1

In [0]:
# Define training data
train_partition = 1

train_path = f"{train_1_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[0 1 2 3 4 5 6 7 8 9]
(45974, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3065/3065 ━━━━━━━━━━━━━━━━━━━━ 4s 940us/step - accuracy: 0.5149 - loss: 14.1082 - val_accuracy: 0.6424 - val_loss: 1.4784
Epoch 2/50
3065/3065 ━━━━━━━━━━━━━━━━━━━━ 3s 871us/step - accuracy: 0.6271 - loss: 1.1548 - val_accuracy: 0.6480 - val_loss: 2.0122
Epoch 3/50
2633/3065 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.6511 - loss: 3.1967

*** WARNING: max output size exceeded, skipping output. ***

3065/3065 ━━━━━━━━━━━━━━━━━━━━ 3s 868us/step - accuracy: 0.7315 - loss: 0.6211 - val_accuracy: 0.7146 - val_loss: 0.6593
Epoch 36/50
3065/3065 ━━━━━━━━━━━━━━━━━━━━ 3s 879us/step - accuracy: 0.7367 - loss: 0.6153 - val_accuracy: 0.7243 - val_loss: 0.6494
Epoch 37/50
3065/3065 ━━━━━━━━━━━━━━━━━━━━ 3s 880us/step - accuracy: 0.7341 - loss: 0.6146 - val_accuracy: 0.7217 - val_loss: 0.6507
Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 32.
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step -

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8832/11690 ━━━━━━━━━━━━━━━━━━━━ 2s

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 868us/step - accuracy: 0.7951 - loss: 0.4974 - val_accuracy: 0.7945 - val_loss: 0.5156
Epoch 17: early stopping
Restoring model weights from the end of the best epoch: 12.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 607us/step - accuracy: 0.7977 - loss: 0.4974
Test accuracy: 79.81%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 607us/step
                precision    recall  f1-score   support

      Analysis     0.8000    0.0400    0.0762       400
      Backdoor     0.8621    0.0716    0.1323       349
           DoS     0.6549    0.0302    0.0577      2453
      Exploits     0.5689    0.9226    0.7038      6679
       Fuzzers     0.7180    0.6833    0.7002      3637
       Generic     0.9990    0.9784    0.9886      8000
        Normal     0.9039    0.8596    0.8812     11200
Reconnaissance     0.8700    0.7717    0.8179      

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3832/3832 ━━━━━━━━━━━━━━━━━━━━ 4s 723us/step - accuracy: 0.5197 - loss: 11.8521
Epoch 2/50
3832/3832 ━━━━━━━━━━━━━━━━━━━━ 3s 731us/step - accuracy: 0.6281 - loss: 0.9171
Epoch 3/50
1175/3832 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/s

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 11s 736us/step - accuracy: 0.7969 - loss: 0.4965


In [0]:
y_pred_4 = model.predict(X_train_)

soft_classification_4 = pd.DataFrame(y_pred_4, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_4['ANN'] = 1
soft_classification_4 = soft_classification_4.reset_index()

y_pred_classes = np.argmax(y_pred_4, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 620us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.8361    0.0876    0.1586      1746
           DoS     0.4598    0.0850    0.1434     12264
      Exploits     0.5675    0.9359    0.7065     33393
       Fuzzers     0.7159    0.7251    0.7205     18184
       Generic     0.9973    0.9790    0.9881     40000
        Normal     0.9350    0.8383    0.8840     56000
Reconnaissance     0.9105    0.7592    0.8280     10491
     Shellcode     0.6432    0.6681    0.6554      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.8011    175341
     macro avg     0.6065    0.5078    0.5085    175341
  weighted avg     0.8076    0.8011    0.7824    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_4_test = model.predict(X_test_)

soft_classification_4_test = pd.DataFrame(y_pred_4_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_4_test['ANN'] = 1
soft_classification_4_test = soft_classification_4_test.reset_index()

y_pred_classes = np.argmax(y_pred_4_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 608us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.0544    0.0326    0.0408       583
           DoS     0.4533    0.0487    0.0879      4089
      Exploits     0.4758    0.9233    0.6280     11132
       Fuzzers     0.2693    0.4685    0.3420      6062
       Generic     0.9980    0.9644    0.9809     18871
        Normal     0.9355    0.6871    0.7923     37000
Reconnaissance     0.8917    0.8009    0.8439      3496
     Shellcode     0.3081    0.6905    0.4261       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7290     82332
     macro avg     0.4386    0.4616    0.4142     82332
  weighted avg     0.7955    0.7290    0.7334     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



path,name,size,modificationTime
dbfs:/mnt/pqsashrd/data/ann_pred_1.csv/,ann_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_1_test.csv/,ann_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_3.csv/,ann_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1.csv/,clust_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1_test.csv/,clust_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3.csv/,clust_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3_test.csv/,clust_pred_3_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/test_1.csv,test_1.csv,38574669,1714569841000
dbfs:/mnt/pqsashrd/data/test_2.csv,test_2.csv,35056701,1714569835000
dbfs:/mnt/pqsashrd/data/test_3.csv,test_3.csv,25575183,1714569796000


#### SAVE PREDICTIONS

In [0]:
dataframes = [soft_classification_0, soft_classification_1, soft_classification_2, soft_classification_3, soft_classification_4]
soft_classification = pd.concat(dataframes, axis=0)

In [0]:
soft_classification[soft_classification["index"]==0]

,index,class_prob_0,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,ANN
0,0,0.000000,4.269195e-23,1.765917e-08,2.627657e-01,1.995735e-05,1.656150e-14,0.737214,0.000000e+00,0.000000,0.000000,0
0,0,0.000000,9.591259e-31,1.932513e-14,1.475097e-18,2.040754e-12,1.857446e-28,1.000000,0.000000e+00,0.000000,0.000000,4
0,0,0.028111,2.471587e-02,1.375131e-01,5.677804e-01,1.877561e-02,3.799237e-02,0.153023,2.412281e-02,0.004167,0.003798,2
0,0,0.000000,1.078010e-19,1.058083e-04,6.639272e-05,3.197152e-02,2.026741e-09,0.967426,2.493328e-17,0.000430,0.000000,3
0,0,0.000000,0.000000e+00,8.304717e-10,1.419650e-07,2.744472e-25,0.000000e+00,1.000000,0.000000e+00,0.000000,0.000000,1


In [0]:
soft_classification = ps.DataFrame(soft_classification).to_spark()
save_path = ann_pred_1_csv_file_path
soft_classification.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)

In [0]:
dataframes = [soft_classification_0_test, soft_classification_1_test, soft_classification_2_test, soft_classification_3_test, soft_classification_4_test]
soft_classification_test = pd.concat(dataframes, axis=0)

In [0]:
soft_classification_test[soft_classification_test["index"]==0]

,index,class_prob_0,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,ANN
0,0,1.065801e-17,7.249567e-04,0.048369,0.151025,0.603468,0.012987,0.183339,0.000087,0.000000,0.000000e+00,0
0,0,8.076419e-04,2.252699e-05,0.038764,0.121870,0.544697,0.021570,0.165498,0.001996,0.104745,2.948706e-05,4
0,0,1.479370e-25,3.156257e-03,0.082727,0.196467,0.287865,0.027751,0.034430,0.003056,0.364227,3.216641e-04,2
0,0,1.867064e-14,5.097497e-07,0.028724,0.067342,0.587155,0.006011,0.171809,0.000003,0.138931,2.501451e-05,3
0,0,3.174885e-11,5.170049e-09,0.083799,0.106818,0.482369,0.012975,0.237457,0.001043,0.075538,7.510422e-16,1


In [0]:
soft_classification_test = ps.DataFrame(soft_classification_test).to_spark()
save_path = ann_pred_1_test_csv_file_path
soft_classification_test.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)